<a href="https://colab.research.google.com/github/RooaaApp/Rooaa/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!git clone https://github.com/pjreddie/darknet
!ls

fatal: destination path 'darknet' already exists and is not an empty directory.
backup	  include	 LICENSE.gen   Makefile		results
cfg	  libdarknet.a	 LICENSE.gpl   obj		scripts
darknet   libdarknet.so  LICENSE.meta  predictions.jpg	src
data	  LICENSE	 LICENSE.mit   python		yolov3.weights
examples  LICENSE.fuck	 LICENSE.v1    README.md


In [0]:
cd darknet

In [0]:
!ls
!sed -i 's/GPU=0/GPU=1/g' Makefile
!make
!wget https://pjreddie.com/media/files/yolov3.weights

In [12]:
cd '/content'

/content


In [17]:
#/////////////////////////////////////////// Constructing Cell //////////////////////////////////////////////////////////////
#-------------------------------------------------------------------------------------------------------------------------------

!pip install opencv-python  
!pip list
!pip install pydub

#-------------------------------------------------------------------------------------------------------------------------------

from google.colab import output
from IPython.display import Audio
from IPython.display import Image
from google.colab import files

#-------------------------------------------------------------------------------------------------------------------------------

import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import time
import random    
import IPython

#-------------------------------------------------------------------------------------------------------------------------------

# load the COCO class labels our YOLO model was trained on
LABELS = open("/content/darknet/data/coco.names").read().strip().split("\n")

# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet("/content/darknet/cfg/yolov3.cfg", "/content/darknet/yolov3.weights")

# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
#-------------------------------------------------------------------------------------------------------------------------------
#Selection of objects (Currently three objects)
def mostDangerousClass(setOfClasses):
    sub = "car"
    res = [s for s in setOfClasses if sub in s]
    if not res:
        sub = "person"
        res = [s for s in setOfClasses if sub in s]
        if not res:
            sub = "chair"
            res = [s for s in setOfClasses if sub in s]
    return res 
    
def mostDangerousSide(leftSide, rightSide):
    sides = set()
    if leftSide:
        sides.add(leftSide[0])
    else: 
        sides.add("")    
    if rightSide:    
        sides.add(rightSide[0])
    else: 
        sides.add("")        
    return mostDangerousClass(sides)

    return mostDangerousClass(sides)
#--------------------------------------------------------------------------------------------------------------------------------

outdoorClasses = [0, 1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 16, 17, 36, 37, 40, 41]
indoorClasses = [0, 16, 17, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 85, 81, 78]
mode = 'indoor'


Package                  Version              
------------------------ ---------------------
absl-py                  0.8.0                
alabaster                0.7.12               
albumentations           0.1.12               
altair                   3.2.0                
astor                    0.8.0                
astropy                  3.0.5                
atari-py                 0.1.15               
atomicwrites             1.3.0                
attrs                    19.1.0               
audioread                2.1.8                
autograd                 1.3                  
Babel                    2.7.0                
backcall                 0.1.0                
backports.tempfile       1.0                  
backports.weakref        1.0.post1            
beautifulsoup4           4.6.3                
bleach                   3.1.0                
blis                     0.2.4                
bokeh                    1.0.4                
boto         

In [18]:
#////////////////////////////////////////////////////Start of the process/////////////////////////////////////////////////////////
#----------------------------------------------------------------------------------------------------------------------
 
#Looping in order to run the cell one time
while True:

  #First: Capturing an Image
  #----------------------------------------------------------------------------------------------------------------------

  try:
    filename = 'download.jpg'
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))


  #----------------------------------------------------------------------------------------------------------------------
  #Second: Use YOLO to analyze the image
  #----------------------------------------------------------------------------------------------------------------------

  imageSource = '/content/'+filename

  img = cv2.imread(imageSource)
  COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")

  # grab the image dimensions and convert it to a blob
  (H, W) = img.shape[:2]
  # construct a blob from the input image and then perform a forward
  # pass of the YOLO object detector, giving us our bounding boxes and
  # associated probabilities
  blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),
  swapRB=True, crop=False)
  net.setInput(blob)
  layerOutputs = net.forward(ln)

  # initialize our lists of detected bounding boxes, confidences, and
  # class IDs, respectively
  boxes = []
  confidences = []
  classIDs = []
  centers = []
  detected= []


  #--------------------------------------------------------------------------------------------------  
  minPer = {0:3.1, 2:6.2, 56:2.1}
  maxPer = {0:12.1, 2:24.2, 56:4.2}
  spacePers = []
  if mode == 'indoor':
    ourClassesIDs = indoorClasses
  elif mode == 'outdoor':
      ourClassesIDs = outdoorClasses

  wrnCenter = set()
  wrnLeft = set()
  wrnRight = set()

  infCenter = set()
  infLeft = set()
  infRight = set()
  #--------------------------------------------------------------------------------------------------

  # loop over each of the layer outputs
  for output in layerOutputs:

    # loop over each of the detections
    for detection in output:

        # extract the class ID and confidence (i.e., probability) of
        # the current object detection
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]

        # filter out weak predictions by ensuring the detected
        # probability is greater than the minimum probability
        #and filter out classes we haven't measure yet    
        if confidence > 0.5 and classID in ourClassesIDs:

            # scale the bounding box coordinates back relative to the
            # size of the image, keeping in mind that YOLO actually
            # returns the center (x, y)-coordinates of the bounding
            # box followed by the boxes' width and height
            box = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY, width, height) = box.astype("int")

            #filter out far objects 
            spacePer = (width * height)/(img.shape[0] * img.shape[1]) * 100  
            spacePers.append(spacePer)
            if spacePer < minPer[classID]: 
              continue

            # use the center (x, y)-coordinates to derive the top and
            # and left corner of the bounding box
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))

            # update our list of bounding box coordinates, confidences,
            # and class IDs
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)
            centers.append((centerX, centerY))
  #--------------------------------------------------------------------------------------------------          

  # apply non-maxima suppression to suppress weak, overlapping
  # bounding boxes
  idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

  texts = []

  # ensure at least one detection exists
  if len(idxs) > 0:

    # loop over the indexes we are keeping
    for i in idxs.flatten():

      # extract the bounding box coordinates
      (x, y) = (boxes[i][0], boxes[i][1])
      (w, h) = (boxes[i][2], boxes[i][3])

      # draw a bounding box rectangle and label on the frame
      color = [int(c) for c in COLORS[classIDs[i]]]
      cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
      text = "{}: {:.4f}".format(LABELS[classIDs[i]],
        confidences[i])
      cv2.putText(img, text, (x, y - 5),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

      label = LABELS[classIDs[i]]
      if label not in detected:
        # find positions and areas of the classes--> (Warning, Informing Areas)
        centerX, centerY = centers[i][0], centers[i][1]

        if centerX <= W/3:
          W_pos = "left "
          if spacePers[i] <= maxPer[classIDs[i]] and spacePers[i] >= minPer[classIDs[i]]:
            infLeft.add(W_pos + label)
          else:
            wrnLeft.add(W_pos + label)
        elif centerX <= (W/3 * 2):
          W_pos = "center "
          if spacePers[i] <= maxPer[classIDs[i]] and spacePers[i] >= minPer[classIDs[i]]:
            infCenter.add(W_pos + label)
          else:
            wrnCenter.add(W_pos + label)
        else:
          W_pos = "right "
          if spacePers[i] <= maxPer[classIDs[i]] and spacePers[i] >= minPer[classIDs[i]]:
            infRight.add(W_pos + label)
          else:
            wrnRight.add(W_pos + label)  

        detected.append(label)

  #--------------------------------------------------------------------------------------------------          
  #obtaining the texts based on:
  #(i)- the most dangerous classes on each location
  #(ii)- the area of the object (Warning Area or Informing Area):
  if wrnCenter or wrnLeft or wrnRight:
    #Warning Area
    texts.append(['watch out!'])
    if wrnCenter:
      texts.append(mostDangerousClass(wrnCenter))
    if wrnLeft or wrnRight:
      texts.append(mostDangerousSide(mostDangerousClass(wrnLeft), mostDangerousClass(wrnRight)))
  else:
    #Informing Area 
    if infCenter:
      texts.append(mostDangerousClass(infCenter))
    if wrnLeft or wrnRight:
      texts.append(mostDangerousSide(mostDangerousClass(infLeft), mostDangerousClass(infRight)))
  if not texts:
    texts.append("Nothing")
  #--------------------------------------------------------------------------------------------------            
  print(texts)
  break
  
  


KeyError: ignored